# Cifar-10 이미지 분류

## 전체 데이터 사용
## data augmentation 적용
## 출처 https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.regularizers import l2
import numpy as np

## 자료형 변환 및 스케일링
- X: 실수형으로 정규화
- Y: 1-hot encoding
- airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

In [4]:
(x_train, y_train0), (x_test, y_test0) = cifar10.load_data()
print(x_train.shape, x_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(x_test.shape, x_test.dtype)
print(y_test0.shape, y_test0.dtype)

(50000, 32, 32, 3) uint8
(50000, 1) uint8
(10000, 32, 32, 3) uint8
(10000, 1) uint8


In [5]:
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

print(x_train.shape, x_train.dtype)

(50000, 32, 32, 3) float32


In [17]:
y_train = tf.keras.utils.to_categorical(y_train0, 10)
y_test = tf.keras.utils.to_categorical(y_test0, 10)
y_train[:4]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

## 모형 구현

In [18]:
np.random.seed(0)
weight_decay = 1e-4

In [19]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_6 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)       

In [20]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001, decay=weight_decay), 
              metrics=['accuracy'])

In [21]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.0003
    return lrate

In [22]:
#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train)

In [23]:
from tensorflow.keras.callbacks import LearningRateScheduler

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=x_train.shape[0] // 64, epochs=50,
                    verbose=2, validation_data=(x_test, y_test),
                    callbacks=[LearningRateScheduler(lr_schedule)])

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 781 steps, validate on 10000 samples
Epoch 1/50
781/781 - 163s - loss: 1.9221 - accuracy: 0.4200 - val_loss: 1.3872 - val_accuracy: 0.5648
Epoch 2/50
781/781 - 162s - loss: 1.2976 - accuracy: 0.5843 - val_loss: 1.1898 - val_accuracy: 0.6410
Epoch 3/50
781/781 - 162s - loss: 1.0914 - accuracy: 0.6493 - val_loss: 1.1207 - val_accuracy: 0.6666
Epoch 4/50
781/781 - 162s - loss: 0.9772 - accuracy: 0.6882 - val_loss: 1.0181 - val_accuracy: 0.6976
Epoch 5/50
781/781 - 164s - loss: 0.9065 - accuracy: 0.7134 - val_loss: 0.8854 - val_accuracy: 0.7315
Epoch 6/50
781/781 - 159s - loss: 0.8537 - accuracy: 0.7345 - val_loss: 0.8734 - val_accuracy: 0.7431
Epoch 7/50
781/781 - 160s - loss: 0.8090 - accuracy: 0.7501 - val_loss: 0.7695 - val_accuracy: 0.7797
Epoch 8/50
781/781 - 162s - loss: 0.7770 - accuracy: 0.7611 - val_loss: 0.8114 - val_accuracy: 0.7668
Epoch 9/50
781/781 - 163s - loss: 0.

KeyboardInterrupt: 

## 모델 평가하기

In [ ]:
cores = model.evaluate(x_test, y_test, batch_size=128, verbose=2)
print('\n Accuracy: %.4f' % scores[1])

## 모뎅 저장하기

In [ ]:
model.save_weights('model/cifar10-full-v3.h5')